In [14]:
import os
import json
import pandas as pd
import numpy as np

# ==============================
# 0. 경로 및 기본 설정
# ==============================

# 140 버전 event log (앞에서 만든 파일)
INPUT_EVENT_LOG_PATH = "./../cohort/cohort_ver140_event_log.csv"

OUTPUT_DIR = "./../cohort"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 최종 출력: cohort ver143
COHORT_143_PATH = os.path.join(OUTPUT_DIR, "cohort_ver143_next_event_arr.csv")

# 너무 짧은 trace 제거 기준
MIN_EVENTS_PER_CASE = 2

# EOS 토큰 ID (다음 이벤트가 없는 마지막 이벤트용)
EOS_EVENT_ID = 0


# ==============================
# 1. 공통 유틸
# ==============================

def _to_datetime(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """주어진 컬럼을 datetime으로 캐스팅."""
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
    return df


# ==============================
# 2. 이벤트 로그 로딩 (140 → raw)
# ==============================

def load_event_log(path: str) -> pd.DataFrame:
    """
    140 버전 Event Log CSV 로딩.
    필수 컬럼:
      - case_id
      - subject_id
      - hadm_id
      - event_name
      - timestamp

    추가로 있다면:
      - death_flag (0/1)
      - dischtime (퇴원 시각)
      - current_heart_rate, current_mean_bp 등도 그대로 사용.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"입력 이벤트 로그 파일을 찾을 수 없습니다: {path}")

    df = pd.read_csv(path)

    required_cols = ["case_id", "subject_id", "hadm_id", "event_name", "timestamp"]
    for c in required_cols:
        if c not in df.columns:
            raise ValueError(
                f"입력 이벤트 로그에 '{c}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}"
            )

    # 시간 컬럼 처리
    df = _to_datetime(df, "timestamp")
    if "dischtime" in df.columns:
        df = _to_datetime(df, "dischtime")

    # timestamp 결측 제거
    df = df.dropna(subset=["timestamp"])

    df = df.sort_values(
        by=["hadm_id", "timestamp", "event_name"]
    ).reset_index(drop=True)

    print(f"[LOAD] Event Log 로딩 완료: {len(df)} rows, {df['hadm_id'].nunique()} hadm_id")
    print(f"[LOAD] event_name 분포:\n{df['event_name'].value_counts()}")
    return df


# ==============================
# 3. Event Log 클린업
#    - (가능하면) ED_ARRIVAL / ED_ARRIVAL_SURR 이후만 사용
#    - DISCHARGE/DEATH 이후 제거
#    - 너무 짧은 trace 제거
# ==============================

def clean_event_log(raw_events: pd.DataFrame,
                    min_events_per_case: int = MIN_EVENTS_PER_CASE) -> pd.DataFrame:
    """
    hadm_id 단위로 다음 규칙 적용:

      1) 시작 기준 이벤트:
         - 우선순위 1: ED_ARRIVAL
         - 우선순위 2: ED_ARRIVAL_SURR
         - 둘 다 없으면: 해당 hadm의 첫 timestamp

      2) DISCHARGE/DEATH 이후 이벤트 제거
         - 둘 다 있으면 더 이른 시점을 기준으로 자름

      3) 남은 이벤트 수가 min_events_per_case 미만이면 제거
    """
    keep_groups = []
    dropped_too_short = 0

    # 통계용 카운트
    cnt_start_ed       = 0  # ED_ARRIVAL 기준 시작
    cnt_start_ed_surr  = 0  # ED_ARRIVAL_SURR 기준 시작
    cnt_start_first    = 0  # 그냥 첫 이벤트 기준 시작

    for hadm_id, g in raw_events.groupby("hadm_id"):
        g = g.sort_values(["timestamp", "event_name"]).copy()
        subject_id = g["subject_id"].iloc[0]
        case_id = g["case_id"].iloc[0]

        # 1) 시작 시각 결정
        is_ed      = (g["event_name"] == "ED_ARRIVAL")
        is_ed_surr = (g["event_name"] == "ED_ARRIVAL_SURR")

        if is_ed.any():
            start_time = g.loc[is_ed, "timestamp"].min()
            cnt_start_ed += 1
        elif is_ed_surr.any():
            start_time = g.loc[is_ed_surr, "timestamp"].min()
            cnt_start_ed_surr += 1
        else:
            start_time = g["timestamp"].min()
            cnt_start_first += 1

        g = g[g["timestamp"] >= start_time].copy()

        # 2) DISCHARGE/DEATH 이후 제거
        is_end = g["event_name"].isin(["DISCHARGE", "DEATH"])
        if is_end.any():
            end_time = g.loc[is_end, "timestamp"].min()
            g = g[g["timestamp"] <= end_time].copy()

        # 3) 최소 이벤트 개수 체크
        if len(g) < min_events_per_case:
            dropped_too_short += 1
            continue

        g["subject_id"] = subject_id
        g["case_id"] = case_id
        keep_groups.append(g)

    if not keep_groups:
        print("[CLEAN] 남아 있는 trace가 없습니다.")
        print(f"[CLEAN] 원본 hadm_id 수: {raw_events['hadm_id'].nunique()}")
        print(f"[CLEAN] 이벤트 수<{min_events_per_case}로 제거된 hadm_id 수: {dropped_too_short}")
        return pd.DataFrame(columns=raw_events.columns)

    clean_df = pd.concat(keep_groups, ignore_index=True)
    clean_df = clean_df.sort_values(
        by=["hadm_id", "timestamp", "event_name"]
    ).reset_index(drop=True)

    print("\n[CLEAN] === 요약 ===")
    print(f"원본 hadm_id 수: {raw_events['hadm_id'].nunique()}")
    print(f"최종 남은 hadm_id 수: {clean_df['hadm_id'].nunique()}")
    print(f"최종 이벤트 row 수: {len(clean_df)}")
    print(f"이벤트 수<{min_events_per_case}로 제거된 hadm_id 수: {dropped_too_short}")
    print("\n[CLEAN] 시작 기준 통계 (hadm 단위):")
    print(f"  ED_ARRIVAL 기준 시작 hadm 수       : {cnt_start_ed}")
    print(f"  ED_ARRIVAL_SURR 기준 시작 hadm 수  : {cnt_start_ed_surr}")
    print(f"  첫 이벤트 기준 시작 hadm 수        : {cnt_start_first}")

    return clean_df


# ==============================
# 4. event_name ↔ event_id 매핑
# ==============================

def build_event_id_map(events: pd.DataFrame) -> pd.DataFrame:
    """
    event_name을 정수 ID로 매핑하는 테이블 생성.
    """
    unique_events = sorted(events["event_name"].unique())
    event_id_map = pd.DataFrame({
        "event_name": unique_events,
        "event_id": range(1, len(unique_events) + 1)
    })
    print(f"[MAP] 이벤트 종류 개수: {len(unique_events)}")
    print(event_id_map)
    return event_id_map


# ==============================
# 5. cohort ver143 (hadm 단위 arr 요약) 생성
# ==============================

def build_cohort_ver143_arr(clean_events: pd.DataFrame,
                            event_id_map: pd.DataFrame) -> pd.DataFrame:
    """
    hadm_id 1건당 1행인 tabular cohort ver143 생성.

    컬럼 개요:
      - subject_id
      - hadm_id
      - case_id
      - n_events
      - death_flag (hadm 단위)
      - dischtime (있으면)
      - first_timestamp, last_timestamp

      + 아래 7개 (요청)
      - prefix_events_str        : 전체 이벤트 이름을 ">"로 이어붙인 문자열
      - current_heart_rate       : 각 이벤트 시점 HR 배열(JSON 문자열)
      - current_mean_bp          : 각 이벤트 시점 BP 배열(JSON 문자열)
      - target_mortality         : hadm 단위 병원 내 사망 여부(0/1)
      - target_next_evt          : 각 이벤트 시점의 다음 이벤트 ID 배열(JSON)
      - target_time_to_next      : 각 이벤트 시점의 다음 이벤트까지 시간(분) 배열(JSON)
      - target_remain_los        : 각 이벤트 시점의 퇴원까지 남은 시간(일) 배열(JSON)
    """
    name_to_id = dict(zip(event_id_map["event_name"], event_id_map["event_id"]))

    clean_events = _to_datetime(clean_events, "timestamp")
    if "dischtime" in clean_events.columns:
        clean_events = _to_datetime(clean_events, "dischtime")

    records = []

    for hadm_id, g in clean_events.groupby("hadm_id"):
        g = g.sort_values(["timestamp", "event_name"]).copy()

        n_events = len(g)
        if n_events < 2:
            # 다음 이벤트가 거의 없기 때문에 스킵할 수도 있음
            continue

        subject_id = g["subject_id"].iloc[0]
        case_id = g["case_id"].iloc[0] if "case_id" in g.columns else hadm_id

        # hadm 단위 death_flag
        if "death_flag" in g.columns:
            df_val = g["death_flag"].iloc[0]
            death_flag = 0 if pd.isna(df_val) else int(df_val)
        else:
            death_flag = 0

        # hadm 단위 dischtime
        disch_time = g["dischtime"].iloc[0] if "dischtime" in g.columns else None

        events = list(g["event_name"])
        times = list(g["timestamp"])
        event_ids = [name_to_id.get(e, -1) for e in events]

        first_timestamp = times[0]
        last_timestamp = times[-1]

        # 1) prefix_events_str: 전체 트레이스
        prefix_events_str = ">".join(events)

        # 2) current_heart_rate 배열
        if "current_heart_rate" in g.columns:
            hr_arr = g["current_heart_rate"].fillna(0).astype(float).tolist()
        else:
            hr_arr = [0.0] * n_events

        # 3) current_mean_bp 배열
        if "current_mean_bp" in g.columns:
            bp_arr = g["current_mean_bp"].fillna(0).astype(float).tolist()
        else:
            bp_arr = [0.0] * n_events

        # 4) target_mortality: hadm 단위 스칼라
        target_mortality = death_flag

        # 5, 6, 7) 다음 이벤트 관련 배열들
        next_evt_ids = []
        time_to_next_arr = []
        remain_los_arr = []

        for i in range(n_events):
            cur_time = times[i]

            # 다음 이벤트 ID, 시간
            if i < n_events - 1:
                next_evt_ids.append(event_ids[i + 1])
                dt_min = (times[i + 1] - cur_time).total_seconds() / 60.0
                time_to_next_arr.append(float(dt_min))
            else:
                # 마지막 이벤트: EOS, 0
                next_evt_ids.append(EOS_EVENT_ID)
                time_to_next_arr.append(0.0)

            # 퇴원까지 남은 시간(일)
            if disch_time is not None and pd.notna(disch_time):
                remain_minutes = (disch_time - cur_time).total_seconds() / 60.0
                remain_days = remain_minutes / (60.0 * 24.0)
                remain_los_arr.append(float(remain_days))
            else:
                remain_los_arr.append(np.nan)

        rec = {
            "subject_id": subject_id,
            "hadm_id": hadm_id,
            "case_id": case_id,
            "n_events": n_events,
            "death_flag": death_flag,
            "dischtime": disch_time,
            "first_timestamp": first_timestamp,
            "last_timestamp": last_timestamp,

            # 요청된 7개 컬럼
            "prefix_events_str": prefix_events_str,
            "current_heart_rate": json.dumps(hr_arr),
            "current_mean_bp": json.dumps(bp_arr),
            "target_mortality": target_mortality,
            "target_next_evt": json.dumps(next_evt_ids),
            "target_time_to_next": json.dumps(time_to_next_arr),
            "target_remain_los": json.dumps(remain_los_arr),
        }

        records.append(rec)

    cohort143 = pd.DataFrame(records)
    print(f"[143] cohort row 수(hadm 기준): {len(cohort143)}")
    return cohort143


# ==============================
# 6. MAIN (cohort ver143 구축)
# ==============================

def main():
    # 1) 140 event log 로딩
    raw_events = load_event_log(INPUT_EVENT_LOG_PATH)

    # 2) Clean: ED/ED_SURR 기준 시작, DISCHARGE/DEATH까지, 너무 짧은 trace 제거
    clean_events = clean_event_log(raw_events, min_events_per_case=MIN_EVENTS_PER_CASE)
    if clean_events.empty:
        print("[MAIN] clean_events가 비어 있습니다. 140 이벤트 생성 로직을 다시 확인하세요.")
        return

    # 3) event_name ↔ event_id 매핑 (메모리에서만 사용)
    event_id_map = build_event_id_map(clean_events)

    # 4) cohort ver143 (hadm 단위 arr 요약) 생성
    cohort143 = build_cohort_ver143_arr(clean_events, event_id_map)

    # 5) 저장 (최종 출력은 ver143 한 개만)
    cohort143.to_csv(COHORT_143_PATH, index=False)
    print(f"[SAVE] cohort ver143 저장 완료: {COHORT_143_PATH}")


if __name__ == "__main__":
    main()


[LOAD] Event Log 로딩 완료: 38674 rows, 1878 hadm_id
[LOAD] event_name 분포:
event_name
ECG_TAKEN         17801
TROP_TAKEN         3778
ANTI_PLT_ADMIN     2860
ANTI_PLT_ORDER     2860
ECG_STEMI_FLAG     2854
ED_ARRIVAL         1869
ED_DEPARTURE       1826
DISCHARGE          1721
ICU_INTIME          943
TROP_POSITIVE       943
ICU_OUTTIME         769
PCI_START           293
DEATH               157
Name: count, dtype: int64

[CLEAN] === 요약 ===
원본 hadm_id 수: 1878
최종 남은 hadm_id 수: 1869
최종 이벤트 row 수: 26860
이벤트 수<2로 제거된 hadm_id 수: 9

[CLEAN] 시작 기준 통계 (hadm 단위):
  ED_ARRIVAL 기준 시작 hadm 수       : 1865
  ED_ARRIVAL_SURR 기준 시작 hadm 수  : 0
  첫 이벤트 기준 시작 hadm 수        : 13
[MAP] 이벤트 종류 개수: 13
        event_name  event_id
0   ANTI_PLT_ADMIN         1
1   ANTI_PLT_ORDER         2
2            DEATH         3
3        DISCHARGE         4
4   ECG_STEMI_FLAG         5
5        ECG_TAKEN         6
6       ED_ARRIVAL         7
7     ED_DEPARTURE         8
8       ICU_INTIME         9
9      ICU_OUTTIME        